# Imports

In [1]:
import sys
import dill as pickle
from math import ceil
import pandas as pd
# to save results to data directory
module_path = '..'
if module_path not in sys.path:
    sys.path.insert(1, module_path)
# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [2]:
import logging
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

# temporarily remove deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

logging.basicConfig(format='%(asctime)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger("TimeSeries")
logger.setLevel(logging.INFO)

output_dir = 'beth_output'

# Dataset

In [3]:
with open(f"_data/BETH/data.pkl", "rb") as fh:
    df= pickle.load(fh)

with open(f"_data/BETH/model.pkl", "rb") as fh:
    outlier_model = pickle.load(fh)

In [4]:
height = 4
width = 6
outlier_key = "userId"
data_f_c = df[outlier_key]
fig = plt.figure()

plt.plot(data_f_c)
plt.xlabel('Time (s)')
plt.ylabel('User ID')

fig.set_size_inches(w=width, h=height)
plt.savefig(f'{output_dir}/base_sig_{outlier_key}_all.pgf')

In [5]:
height = 4
width = 6
# In progress Plot
fig = plt.figure()
plt.plot(outlier_model.max_val,'b-', label='max')
plt.plot( outlier_model.max_mean,'r-',label=r'$\mu$')
plt.plot( outlier_model.max_std_dev,'g-',label=r'$\sigma$')
plt.xlabel('time')
plt.legend(loc=0)

fig.set_size_inches(w=width, h=height)
plt.savefig(f'{output_dir}/mp_hist_{outlier_key}.pgf')

In [6]:
# Results plot
fig = plt.figure()
data_test = df.copy()
plt.plot(data_test["evil"] )
# plt.title(f"Detection Method: Anomaly")
logger.info(outlier_model.anomalies)
for i in outlier_model.anomalies:
    plt.axvline(x=i,color='r')
plt.xlabel(r'time')
plt.ylabel(r'evil')

fig.set_size_inches(w=width, h=height)
plt.savefig(f'{output_dir}/outlier_result_evil_{outlier_key}.pgf')

04/22/2022 12:03:47 PM: [11000, 213090, 237204, 485354, 787618, 816542]


In [7]:
# Results plot
fig = plt.figure()
data_test = df.copy()
plt.plot(data_test["sus"] )
# plt.title(f"Detection Method: Anomaly")
logger.info(outlier_model.anomalies)
for i in outlier_model.anomalies:
    plt.axvline(x=i,color='r')
plt.xlabel(r'time')
plt.ylabel(r'sus')

fig.set_size_inches(w=width, h=height)
plt.savefig(f'{output_dir}/outlier_result_sus_{outlier_key}.pgf')

04/22/2022 12:03:47 PM: [11000, 213090, 237204, 485354, 787618, 816542]
